# Retinal Organoid Stim (ResponsiveNeurons)

- Digital data of light pulses (extracted by spikeinterface) is extracted and processed
- This Jupyter Notebook is complete note book that extracts and categorize data of baseline and stimulation from .CSV extracts of spike interface.
- The first steps of this notebook is dealing with merging the stim timestamps with unit timeseries 
- Then the code will make data ready for NEX burst, PSTH and FiringHisto analysis
- The results of NEX is later imported to categorize, plot and make it ready for Prism statistics

In [ ]:
## Step 01 _ Read txt file of stim Digit file and extract the pulse-start and pulse-end times
import os
import pandas as pd

def extract_stimulus_events(txt_file):
    stim_on = []  # List to store stimulus onset times
    stim_off = []  # List to store stimulus offset times
    
    with open(txt_file, 'r') as file:
        lines = file.readlines()
        lines = lines[2:]
        
        for i in range(1, len(lines)):  # Start from the second row
            try:
                prev_value = int(lines[i-1].split()[1])
                curr_value = int(lines[i].split()[1])
                if prev_value == 0 and curr_value == 4:
                    stim_on.append(float(lines[i].split()[0]) / 1000)  # Convert to seconds
                elif prev_value == 4 and curr_value == 0:
                    stim_off.append(float(lines[i].split()[0]) / 1000)  # Convert to seconds
            except (ValueError, IndexError):
                continue  
    
    return stim_on, stim_off

def export_to_excel(txt_file, stim_on, stim_off):
    file_name = os.path.splitext(txt_file)[0]
    df = pd.DataFrame({'Stim_ON': stim_on, 'Stim_OFF': stim_off})
    output_file = f"{file_name}_stimulus_times.xlsx"
    df.to_excel(output_file, index=False)
    return output_file

def split_data_and_save_to_excel(input_file, output_file):
    df = pd.read_excel(input_file)
    categories = ['50ms_0.2Hz', '50ms_1.0Hz', '50ms_2.0Hz', '50ms_5.0Hz']
    start_indices = [2, 7, 17, 37] # This depends on light stim protocol (e.g. pulse 2 to 6 is 50ms_0.5Hz)
    end_indices = [6, 16, 36, 61]  
    
    with pd.ExcelWriter(output_file) as writer:
        for cat, start, end in zip(categories, start_indices, end_indices):
            cat_df = df.iloc[start-2:end-1]  
            cat_df.to_excel(writer, sheet_name=cat, index=False)

def process_files_and_split_data(txt_folder):
    for txt_file in os.listdir(txt_folder):
        if txt_file.endswith(".txt"):
            txt_path = os.path.join(txt_folder, txt_file)
            stim_on, stim_off = extract_stimulus_events(txt_path)
            excel_file = export_to_excel(txt_path, stim_on, stim_off)
            split_data_and_save_to_excel(excel_file, f"{os.path.splitext(txt_path)[0]}_splitted.xlsx")
            print(f"Processed file: {txt_file}")

# Example usage:
txt_folder = r'C:\StimTimes_Digital_Txt' 
process_files_and_split_data(txt_folder)

In [ ]:
## Merge Pulse timestamps with neuronal timestamp file
import pandas as pd
import os

def extract_data_from_excel(excel_file):
    sheets_data = pd.read_excel(excel_file, sheet_name=None)
    
    extracted_data = {}
    for sheet_name, sheet_df in sheets_data.items():
        extracted_data[sheet_name] = sheet_df.iloc[:, :2]
    return extracted_data
def add_sheet_names_to_columns(data):
    for sheet_name, sheet_data in data.items():
        new_column_names = [f"{sheet_name}_{col}" for col in sheet_data.columns]
        sheet_data.columns = new_column_names
    return data
def merge_excel_and_csv(excel_data, csv_file):
    csv_data = pd.read_csv(csv_file)
    merged_data = pd.concat([pd.concat(excel_data.values(), axis=1), csv_data], axis=1)
    return merged_data
def save_to_csv(output_csv_file, merged_data):
    merged_data.to_csv(output_csv_file, index=False)

# Load pulse timestamp file and unit timestamp file
excel_folder = r"C:\StimTimestamps" 
csv_folder = r"C:\UnitTimeseries"  
output_folder = r"C:\Stim-Unit_Timestamps"  
os.makedirs(output_folder, exist_ok=True)

for excel_file in os.listdir(excel_folder):
    if excel_file.endswith(".xlsx"):
        excel_path = os.path.join(excel_folder, excel_file)
        extracted_data = extract_data_from_excel(excel_path)
        add_sheet_names_to_columns(extracted_data)
        
        for csv_file in os.listdir(csv_folder):
            if csv_file.endswith(".csv") and excel_file[:28] in csv_file:
                csv_path = os.path.join(csv_folder, csv_file)
                merged_data = merge_excel_and_csv(extracted_data, csv_path)
                output_csv_file_name = os.path.splitext(csv_file)[0] + "_merged.csv"
                output_csv_file_path = os.path.join(output_folder, output_csv_file_name)
                save_to_csv(output_csv_file_path, merged_data)

In [ ]:
## 1) convert the timestapm data into seconds
## 2) Adds 50ms or 0.05 seconds to the Stim_OFF time stamps --> the ON time will be ON + (50ms of OFF time)
## 2) Concatinate all ON and OFF time stamp data into two columns 

import os
import pandas as pd

def adjust_and_concatenate_columns(csv_file, sampling_frequency):
    df = pd.read_csv(csv_file)
    df = df / sampling_frequency
    column_a_data = []
    column_b_data = []
    for i in range(0, 14, 2):
        column_a_data.extend(df.iloc[:, i].dropna().values)
        column_b_data.extend(df.iloc[:, i+1].dropna().values)
    column_b_data = [x + 0.05 for x in column_b_data]
    new_df = pd.DataFrame({
        'Stim_ON': column_a_data,
        'Stim_OFF': column_b_data
    })
    
    remaining_columns = df.iloc[:, 14:]
    max_rows = max(new_df.shape[0], remaining_columns.shape[0])
    new_df = new_df.reindex(range(max_rows))
    remaining_columns = remaining_columns.reindex(range(max_rows))
    final_df = pd.concat([new_df, remaining_columns], axis=1)
    return final_df

def process_files_and_save_to_csv(csv_folder, output_folder, sampling_frequency):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith(".csv"):
            csv_path = os.path.join(csv_folder, csv_file)
            df_adjusted = adjust_and_concatenate_columns(csv_path, sampling_frequency)
            output_file_name = os.path.splitext(csv_file)[0] + "_stimDivSampFreq_ONOFF0.05.csv"
            output_file = os.path.join(output_folder, output_file_name)
            df_adjusted.to_csv(output_file, index=False)

csv_folder = r"C:\Vasc_Stim"  
output_folder = r"C:\Stim_SampFreqAdj"  
sampling_frequency = 32000  # Specify the sampling frequency

In [ ]:
## 
import os
import pandas as pd

source_folder = r'C:\Stim_SampFreqAdj'
output_excel_path = r'C:\Stim_SamFreqAdj_Freqcat.xlsx'

def generate_frequency_pulse_width():
    frequency = []
    pulse_width = []
    frequency.extend([0.5] * 5)
    frequency.extend([1] * 10)
    frequency.extend([2] * 20)
    frequency.extend([5] * 25)
    pulse_width.extend([50] * 60)
    return frequency, pulse_width

def process_csv_files(source_folder, output_excel_path):
    with pd.ExcelWriter(output_excel_path, engine='xlsxwriter') as writer:
        for file_name in os.listdir(source_folder):
            if file_name.endswith('.csv'):
                file_path = os.path.join(source_folder, file_name)
                df = pd.read_csv(file_path)
                frequency, pulse_width = generate_frequency_pulse_width()
                new_columns_df = pd.DataFrame({
                    'Frequency': frequency,
                    'Pulse_width': pulse_width
                })
                combined_df = pd.concat([new_columns_df, df], axis=1)
                sheet_name = os.path.splitext(file_name)[0]  
                combined_df.to_excel(writer, sheet_name=sheet_name, index=False)
# Run the process
process_csv_files(source_folder, output_excel_path)

In [ ]:
## N of APs per ON and OFF
import pandas as pd

input_excel_path = r'C:\Stim_SamFreqAdj_Freqcat.xlsx'
output_excel_path = r'C:\ON-OFF.xlsx'

def process_sheet(df):
    frequency = df['Frequency']
    pulse_width = df['Pulse_width']
    stim_on = df['Stim_ON'].dropna().values
    stim_off = df['Stim_OFF'].dropna().values
    neuronal_data = df.iloc[:, 4:]
    neuron_names = neuronal_data.columns
    on_off_results = {neuron_name: [] for neuron_name in neuron_names}
    all_stim_on = []
    all_stim_off = []
    for on, off in zip(stim_on, stim_off):
        adjusted_off = off + 0.05 
        all_stim_on.append(on)
        all_stim_off.append(adjusted_off)
        for neuron_name in neuron_names:
            neuron_timestamps = neuronal_data[neuron_name].dropna().values
            count_within = sum(on <= ts <= adjusted_off for ts in neuron_timestamps)
            on_off_results[neuron_name].append(count_within)
    
    on_off_data = pd.DataFrame(on_off_results)
    on_off_data.insert(0, 'Stim_ON', all_stim_on)
    on_off_data.insert(1, 'Stim_OFF', all_stim_off)
    on_off_data.insert(0, 'Pulse_width', pulse_width)
    on_off_data.insert(0, 'Frequency', frequency)
    
    off_next_on_results = {neuron_name: [] for neuron_name in neuron_names}
    all_stim_on_next = []
    all_stim_off_current = []
    for i in range(len(stim_off)):
        adjusted_off = stim_off[i] + 0.05 
        next_on_time = stim_on[i + 1] if (i + 1) < len(stim_on) else None
        
        if next_on_time:
            all_stim_on_next.append(next_on_time)
            all_stim_off_current.append(adjusted_off)
            for neuron_name in neuron_names:
                neuron_timestamps = neuronal_data[neuron_name].dropna().values
                count_within = sum(adjusted_off <= ts <= next_on_time for ts in neuron_timestamps)
                off_next_on_results[neuron_name].append(count_within)
    
    # Convert OFF-Next ON results to DataFrame
    off_next_on_data = pd.DataFrame(off_next_on_results)
    off_next_on_data.insert(0, 'Stim_ON', all_stim_on_next)
    off_next_on_data.insert(1, 'Stim_OFF', all_stim_off_current)
    off_next_on_data.insert(0, 'Pulse_width', pulse_width)
    off_next_on_data.insert(0, 'Frequency', frequency)
    
    return on_off_data, off_next_on_data

excel_data = pd.ExcelFile(input_excel_path)
with pd.ExcelWriter(output_excel_path, engine='xlsxwriter') as writer:
    for sheet_name in excel_data.sheet_names:
        df = pd.read_excel(excel_data, sheet_name=sheet_name)
        on_off_data, off_next_on_data = process_sheet(df)
        on_off_data.to_excel(writer, sheet_name=f'{sheet_name}_ON', index=False)
        off_next_on_data.to_excel(writer, sheet_name=f'{sheet_name}_OFF', index=False)

In [ ]:
## Freq per ON and OFF periods
import pandas as pd
import os

input_excel_path = r'C:\ON-OFF.xlsx'
output_excel_path = r'C:\ON-OFF_Freq.xlsx'

def process_sheet(df):
    frequency = df['Frequency']
    pulse_width = df['Pulse_width']
    stim_on = df['Stim_ON'].dropna().values
    stim_off = df['Stim_OFF'].dropna().values
    stim_delta = abs(stim_on - stim_off)
    neuronal_data = df.iloc[:, 4:]
    for i in range(len(neuronal_data.columns)):
        neuronal_data.iloc[:, i] = neuronal_data.iloc[:, i] / stim_delta
    processed_df = pd.concat([frequency, pulse_width, df[['Stim_ON', 'Stim_OFF']], neuronal_data], axis=1)

    return processed_df

excel_data = pd.ExcelFile(input_excel_path)
with pd.ExcelWriter(output_excel_path, engine='xlsxwriter') as writer:
    for sheet_name in excel_data.sheet_names:
        df = pd.read_excel(excel_data, sheet_name=sheet_name)
        processed_df = process_sheet(df)
        processed_df.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
## Determine Responsive and non-responsive Neurons
import pandas as pd
import numpy as np

input_excel_path = r'C:\ON-OFF_Freq.xlsx'
output_excel_path = r'C:\Responding_norm.xlsx'

def sd(data):
    return np.std(data, ddof=1)

def process_neurons(on_df, off_df):
    frequencies = on_df['Frequency'].unique()
    res_data = {'Frequency': frequencies}
    non_res_data = {'Frequency': frequencies}

    for neuron in on_df.columns[4:]:
        on_means = on_df.groupby('Frequency')[neuron].mean()
        off_means = off_df.groupby('Frequency')[neuron].mean()
        off_sds = off_df.groupby('Frequency')[neuron].apply(sd)
        off_means_replaced = off_means.replace(0, 0.1)
        normalized_values = on_means / off_means_replaced
        res_count = (on_means > off_means + (2 * off_sds)).sum()
        is_res = res_count >= 2
        if is_res:
            res_data[neuron] = normalized_values
        else:
            non_res_data[neuron] = normalized_values
    res_df = pd.DataFrame(res_data)
    non_res_df = pd.DataFrame(non_res_data)
    return res_df, non_res_df

excel_data = pd.ExcelFile(input_excel_path)
with pd.ExcelWriter(output_excel_path, engine='xlsxwriter') as writer:
    for sheet_name in excel_data.sheet_names:
        if sheet_name.endswith('_ON'):
            base_name = sheet_name[:-3]
            off_sheet_name = f'{base_name}_OFF'
            if off_sheet_name in excel_data.sheet_names:
                on_df = pd.read_excel(excel_data, sheet_name=sheet_name)
                off_df = pd.read_excel(excel_data, sheet_name=off_sheet_name)
                res_df, non_res_df = process_neurons(on_df, off_df)
                res_sheet_name = f'{base_name}_Res'
                non_res_sheet_name = f'{base_name}_Non_Res'
                res_df.to_excel(writer, sheet_name=res_sheet_name, index=False)
                non_res_df.to_excel(writer, sheet_name=non_res_sheet_name, index=False)

In [ ]:
## Categorization of data
import pandas as pd
import numpy as np
import os

input_excel_path = r'C:\Responding_Norma.xlsx'
output_excel_path = r'C:\Responding_Summarized.xlsx'

def sem(data):
    return np.std(data, ddof=1) / np.sqrt(len(data))

def process_sheet(sheet_name, sheet_df):
    parts = sheet_name.split('_')
    group = parts[1]  # Vasc or NonV
    week = parts[0]  # w17, w20, or w23
    neuron_type = parts[4]  # Res or Non_Res
    pulse_intensity = parts[3]  # _02_, _03_, or _04_

    output_data = {
        'Group': [],
        'Week': [],
        'Neuron_Type': [],
        'Pulse_Intensity': [],
        'Sample_Name': [],
        'Frequency': [],
        'Average': [],
        'Number': [],
        'Max': [],
        'Min': [],
        'Median': [],
        'SDV': [],
        'SEM': []
    }

    for frequency in sheet_df['Frequency'].unique():
        freq_df = sheet_df[sheet_df['Frequency'] == frequency]
        neuron_data = freq_df.iloc[:, 1:].dropna(axis=1, how='all')
        if neuron_data.shape[1] == 0:
            avg = 0
            n = 0
            max_val = 0
            min_val = 0
            median = 0
            sdv = 0
            sem_val = 0
        else:
            avg = neuron_data.mean(axis=1).mean()
            n = neuron_data.shape[1]
            max_val = neuron_data.max(axis=1).max()
            min_val = neuron_data.min(axis=1).min()
            median = neuron_data.median(axis=1).median()
            sdv = neuron_data.std(axis=1).mean()
            sem_val = neuron_data.apply(sem, axis=1).mean()

        output_data['Group'].append(group)
        output_data['Week'].append(week)
        output_data['Neuron_Type'].append(neuron_type)
        output_data['Pulse_Intensity'].append(pulse_intensity)
        output_data['Sample_Name'].append(sheet_name)
        output_data['Frequency'].append(frequency)
        output_data['Average'].append(avg)
        output_data['Number'].append(n)
        output_data['Max'].append(max_val)
        output_data['Min'].append(min_val)
        output_data['Median'].append(median)
        output_data['SDV'].append(sdv)
        output_data['SEM'].append(sem_val)

    return pd.DataFrame(output_data)

excel_data = pd.ExcelFile(input_excel_path)
all_data = []
for sheet_name in excel_data.sheet_names:
    sheet_df = pd.read_excel(excel_data, sheet_name=sheet_name)
    processed_df = process_sheet(sheet_name, sheet_df)
    all_data.append(processed_df)
summary_df = pd.concat(all_data, ignore_index=True)
summary_df.to_excel(output_excel_path, index=False)

In [ ]:
## Remove outliers, plot, save for Prism analysis
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

colors = {
    'Non_Vasc_NonRespond': 'gray',
    'Vasc_NonRespond': 'lightcoral',
    'Non_Vasc_Respond': '#B88B72',  # A color between gray (#808080) and orange (#FFA500)
    'Vasc_Respond': '#D4A017'  # A color between lightcoral (#F08080) and orange (#FFA500)
}

file_path = r'C:\Responding_Summarized.xlsx'
xls = pd.ExcelFile(file_path)

def remove_outliers(df, column):
    def _remove_outliers(group):
        Q1 = group[column].quantile(0.25)
        Q3 = group[column].quantile(0.75)
        IQR = Q3 - Q1
        return group[~((group[column] < (Q1 - 3 * IQR)) | (group[column] > (Q3 + 5 * IQR)))]
    return df.groupby(['Group', 'Week']).apply(_remove_outliers).reset_index(drop=True)

def calculate_group_statistics(df):
    group_stats = df.groupby(['Week', 'Group'])['Average'].agg(['mean', 'std', 'sem', 'max', 'min', 'count']).reset_index()
    # Calculate number of data points per group
    group_counts = df.groupby('Group')['Average'].count().reset_index()
    group_stats['Count'] = group_counts['Average']
    
    return group_stats
processed_data = {}
variable_stats_raw = {}
variable_stats_cleaned = {}
for sheet_name in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name)
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Week', 'Average'])
    group_stats_raw = calculate_group_statistics(df)
    variable_stats_raw[sheet_name] = group_stats_raw
    # --------------------- Remove outliers --------------------
    df_cleaned = remove_outliers(df, 'Average')
    processed_data[sheet_name] = df_cleaned
    # Save the cleaned data to a new Excel file
    #cleaned_excel_path = f"{output_directory}/{sheet_name}_OutliersRemoved.xlsx"
    #df_cleaned.to_excel(cleaned_excel_path, index=False)
    # Calculate statistics per group
    group_stats = calculate_group_statistics(df_cleaned)
    variable_stats_cleaned[sheet_name] = group_stats
    group_order = ['Non_Vasc_NonRespond', 'Vasc_NonRespond', 'Non_Vasc_Respond', 'Vasc_Respond']
    # Plot using seaborn with custom colors
    plt.figure(figsize=(7, 5))  # Increase figure size
    ax=sns.boxplot(data=df_cleaned, x='Week', y='Average', hue='Group', 
                order=['w17', 'w20', 'w23'],
                hue_order=group_order, palette=colors, showfliers=False, linewidth=0.5)  
    sns.stripplot(data=df_cleaned, x='Week', y='Average', hue='Group', 
                  order=['w17', 'w20', 'w23'],
                  hue_order=group_order, dodge=True, jitter=True, edgecolor='black', linewidth=0.5, palette=colors, 
                  alpha=0.5, size=2.5)  
    means = df_cleaned.groupby(['Week', 'Group'])['Average'].mean().reset_index()
    for group, color in colors.items():
        plt.plot(means[means['Group'] == group]['Week'], means[means['Group'] == group]['Average'], 
                 color=color, linestyle='-', linewidth=1, marker='o', markersize=5, markeredgecolor='black',
                 markeredgewidth=1,zorder=10 )  # Add markers
    for group, color in colors.items():
        sems = df_cleaned.groupby(['Week', 'Group'])['Average'].sem().reset_index()
        group_sems = sems[sems['Group'] == group]
        for dpi in ['w17', 'w20', 'w23']:
            dpi_sems = group_sems[group_sems['Week'] == dpi]
            plt.errorbar(dpi_sems['Week'], means[(means['Group'] == group) & (means['Week'] == dpi)]['Average'], 
                         yerr=dpi_sems['Average'], fmt='o', color=color, markersize=8, capsize=3, 
                         markeredgecolor='black', linewidth=1, zorder=10)  # Add error bars with black color and adjustable line width
    plt.xticks(rotation=45, fontsize=18)
    ax.set_yscale('log')
    ax.set_ylim(1e-2, 1e2)  
    plt.legend(title='Group', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xlabel('Week', fontsize=16)  
    plt.ylabel('Firing Frequency (Hz)', fontsize=18)  
    plt.title(f'{sheet_name}', fontsize=18)  
    plt.yticks(fontsize=16)  
    plt.tight_layout()
    
    #output_directory= r'C:\.....'
    # Save the plot as a PDF file
    #pdf_filename = f"{output_directory}/{sheet_name}_VascNonVasc_Weeks.pdf"
    #plt.savefig(pdf_filename, format='pdf')
    plt.show()  